# Steem Mentions

In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import math
from decimal import Decimal
import tabulate
from constants import MONGO_URL_DATA_FILE_PATH, MONGO_URL_COLUMNS, AUTHOR_DATA_FILE_PATH
MONGO_URL_DATA_FILE_PATH

'/home/ubuntu/steem-mentions/./data/mongo_url_list.csv'

## Loading Data

### Url data

In [2]:
url_data = pd.DataFrame.from_csv(MONGO_URL_DATA_FILE_PATH)

In [3]:
url_data.head()

,net_votes,total_payout_value,created,url,host_name,domain,author,parent_author,parent_permlink,permlink
id,,,,,,,,,,
4,6,1.024,2016-04-06 19:54:12,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1
19,23,81.774,2016-04-08 15:32:57,https://github.com/wackou,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://github.com/wackou/bts_tools,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://bts-tools.readthedocs.io/,bts-tools.readthedocs.io,readthedocs.io,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://steemit.com/steem/@wackou/btstools-rel...,steemit.com,steemit.com,wackou,NaN,witness-category,wackou-witness-post


In [4]:
len(url_data)

10156778

### Author data

In [5]:
author_data = pd.DataFrame.from_csv(AUTHOR_DATA_FILE_PATH)
author_data.head()

,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests
name,,,,,,,
itcanbeprevented,2016-08-13 22:59:00,2016-09-05 22:48:48,2016-09-05 22:48:48,71465443,0.000,0.000,1.090607e+04
shootit,2017-07-17 17:10:48,2017-08-19 16:57:57,2017-08-19 16:56:33,68361015,0.000,0.000,1.033498e+03
pratibha,2017-08-09 07:30:09,2017-09-26 15:55:21,2017-09-19 20:22:51,7121429731,0.005,0.000,3.514357e+03
lilytran,2016-07-23 05:01:27,2016-08-17 21:54:42,2016-08-18 20:54:15,8370720214,0.000,0.123,1.304985e+04
exyle,2016-07-14 07:08:00,2017-09-30 18:33:33,2017-09-30 18:27:15,115863519050039,0.000,2578.350,1.861507e+08


Calculate reputation score
https://steemit.com/steemit/@digitalnotvir/how-reputation-scores-are-calculated-the-details-explained-with-simple-math

In [6]:
def reputation_score(reputation):
    neg = False if reputation >= 0 else True
    if reputation == 0:
        out = 0
    else:
        out = math.log(math.fabs(reputation), 10)
    out = max(out - 9, 0)
    out = out * -1 if neg else out
    out = (out * 9) + 25
    return math.floor(out)

In [7]:
author_data.iloc[39]

created           2017-08-02 08:09:30
last_vote_time    2017-09-15 13:25:24
last_post         2017-08-24 19:30:15
reputation                14520822935
total_steem                     0.001
total_sbd                        4.56
total_vests                   19548.4
Name: geopoliticsnow, dtype: object

In [8]:
reputation_score(author_data.iloc[39]['reputation'])

35

In [9]:
author_data.loc["leyroft"]

created           2017-08-05 04:49:06
last_vote_time    2017-09-30 13:46:57
last_post         2017-08-15 06:08:12
reputation                          0
total_steem                         0
total_sbd                       0.001
total_vests                   1327.21
Name: leyroft, dtype: object

In [10]:
author_data['reputation_score'] = author_data.apply(lambda row: reputation_score(row['reputation']), axis=1)

Add a numerical index for authors

In [11]:
author_data.reset_index(inplace=True)

In [12]:
author_data.head()

,name,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,itcanbeprevented,2016-08-13 22:59:00,2016-09-05 22:48:48,2016-09-05 22:48:48,71465443,0.000,0.000,1.090607e+04,25
1,shootit,2017-07-17 17:10:48,2017-08-19 16:57:57,2017-08-19 16:56:33,68361015,0.000,0.000,1.033498e+03,25
2,pratibha,2017-08-09 07:30:09,2017-09-26 15:55:21,2017-09-19 20:22:51,7121429731,0.005,0.000,3.514357e+03,32
3,lilytran,2016-07-23 05:01:27,2016-08-17 21:54:42,2016-08-18 20:54:15,8370720214,0.000,0.123,1.304985e+04,33
4,exyle,2016-07-14 07:08:00,2017-09-30 18:33:33,2017-09-30 18:27:15,115863519050039,0.000,2578.350,1.861507e+08,70


### Urls with author data

In [13]:
url_with_author_data = pd.merge(url_data, author_data, left_on='author', right_on="name")
url_with_author_data.head()

,net_votes,total_payout_value,created_x,url,host_name,domain,author,parent_author,parent_permlink,permlink,name,created_y,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,6,1.024,2016-04-06 19:54:12,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
1,-1,0.000,2016-04-12 20:13:18,http://www.washingtonpost.com/wp-srv/national/...,www.washingtonpost.com,washingtonpost.com,red,NaN,spam,spamtford,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
2,0,0.000,2016-04-12 20:54:24,https://www.youtube.com/watch?v=B0N1gQXETmU,www.youtube.com,youtube.com,red,NaN,Does voting even matter?,newswearechange,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
3,0,0.000,2016-04-16 02:58:42,http://blog.tenthamendmentcenter.com/2016/04/s...,blog.tenthamendmentcenter.com,tenthamendmentcenter.com,red,NaN,news,governor-mississippi-constitutional-carry-law-...,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
4,1,0.184,2016-04-16 03:33:39,https://scontent-ord1-1.xx.fbcdn.net/hphotos-x...,scontent-ord1-1.xx.fbcdn.net,fbcdn.net,red,NaN,news,laughable,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54


### Truncate settings

In [14]:
pd.set_option('display.max_colwidth', -1)

Truncate the url data for testing

In [15]:
use_truncate = False
if use_truncate:
    url_with_author_data = url_with_author_data[0:50000]
    url_data = url_data[0:50000]

### Group by with metrics

In [17]:
def group_on_field_with_metrics(field_name):
    grouped = url_with_author_data.groupby(field_name, as_index=False)
    url_authors = grouped['author'].agg({'mentions': 'count', 'num_authors': 'nunique','authors': lambda x: np.unique(x)})
    url_votes = grouped['net_votes'].agg({'total_votes': 'sum', 'avg_votes': 'mean'})
    url_payouts = grouped['total_payout_value'].agg({'total_payout': 'sum', 'avg_payout': 'mean'})
    url_reputation = grouped['reputation_score'].agg({'total_reputation': 'sum', 'avg_reputation': 'mean'})
    merge_left = pd.merge(url_authors, url_votes)
    merge_left = pd.merge(merge_left, url_payouts)
    unique_totals = pd.merge(merge_left, url_reputation)
    return unique_totals

### Unique urls with metrics data

Let's make a totals data frame for unique urls

In [18]:
grouped = url_with_author_data.groupby('url', as_index=False)

In [19]:
url_authors = grouped['author'].agg({'mentions': 'count', 'num_authors': 'nunique','authors': lambda x: np.unique(x)})

In [20]:
url_votes = grouped['net_votes'].agg({'total_votes': 'sum', 'avg_votes': 'mean'})    
#url_votes = grouped['net_votes'].agg({'total_votes': lambda x: np.sum(x), 'avg_votes': lambda x: np.sum(x) / len(x)})    

In [21]:
url_payouts = grouped['total_payout_value'].agg({'total_payout': 'sum', 'avg_payout': 'mean'})

In [22]:
url_reputation = grouped['reputation_score'].agg({'total_reputation': 'sum', 'avg_reputation': 'mean'})

In [23]:
merge_left = pd.merge(url_authors, url_votes)
merge_left = pd.merge(merge_left, url_payouts)
unique_urls = pd.merge(merge_left, url_reputation)

In [24]:


#unique_urls = group_on_field_with_metrics('url')
# add domain
url_domains = grouped['domain'].agg({'domain': lambda x: np.unique(x)})
unique_urls = pd.merge(unique_urls, url_domains)
unique_urls.head()

,url,num_authors,mentions,authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation,domain
0,http://![Antet.jpg,1,1,mybestdeal4you,1,1.0,0.000,0.000,25.0,25,jpg.
1,http://![IMG-20170929-WA0000.jpg,1,1,sitinisa12,0,0.0,0.000,0.000,24.0,24,jpg.
2,http://![PicsArt_05-17-04.05.22.jpg,1,1,obasco,6,6.0,0.182,0.182,33.0,33,jpg.
3,http://![Using-Water-as-a-Medicine-Drinking-Water-On-Empty-Stomach.jpg,1,1,sajibkumar,5,5.0,0.000,0.000,25.0,25,jpg.
4,http://![btcc.jpg,1,1,manueldel,0,0.0,0.000,0.000,25.0,25,jpg.


### Unique domains with metric data

In [39]:
unique_domains = group_on_field_with_metrics('domain')
unique_domains.head()

,domain,num_authors,mentions,authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation
0,!.,1,1,bennywb56,4,4.000000,0.000,0.000000,45.000000,45
1,![.,7,7,"[alexzicky, awaismuneeb1, eosio, omojesus, pnicoll, shariff, ultimatepeace]",288,41.142857,0.022,0.003143,40.285714,282
2,![imag*italic*e.,1,1,fontenotjohn,1,1.000000,0.000,0.000000,25.000000,25
3,""",.",2,2,"[nkdk, rnason]",14,7.000000,0.388,0.194000,54.500000,109
4,""".",22,36,"[ace108, afrog, benthegameboy, billionairesmind, capt, carlosleyvah, catweasel, dots, elewarne, evimeria, facemaker77, faddat, hagie, icaruspony, joexherph, kraizmancom, ninjavz, steemblake, steemvoter, steve1122, valhalla-hash, vinz0380]",2421,67.250000,1314.049,36.501361,51.833333,1866


## URL's

### Most mentioned

In [40]:
unique_urls.sort_values('mentions', ascending=False).head()

,url,num_authors,mentions,authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation,domain
3431336,https://steemit.com/@hr1,9,8620,"[cryptomonitor, dtworker, enriqueig, helphand, masteryoda, naughtyhrn, rgeddes, srtechtube, steemerhrn]",170326,19.759397,7116.951,0.825632,60.149304,518487,steemit.com
4406043,https://steemitimages.com/DQmRKgYYp1TzWmvqtnfbMSLZQSgUXinUxqyHyd39HZ8j7gx/upvote.gif,15,7806,"[aek081969, algeriatech, braini, cloudculpepper, correctbro, cryptomonitor, da-primate, danmax, goldcoin, mannyfig1956, monkrypto, raoharoon, rezarezz, unitedstate, zf90]",161069,20.633999,1495.971,0.191644,60.322316,470876,steemitimages.com
3426465,https://steemit.com/@dtworker,4,7773,"[cryptomonitor, dtworker, naughtyhrn, steemerhrn]",129519,16.662679,1255.188,0.161481,59.624855,463464,steemit.com
3424405,https://steemit.com/@cryptomonitor,2,7657,"[asdrenbeqiri, cryptomonitor]",160700,20.987332,1461.644,0.190890,60.996213,467048,steemit.com
3421378,https://steemit.com/@blocktrades,19,7415,"[aboundlessworld, alfa-good, cryptomonitor, cryptovest, cyberanalytics, dailysteemnews, dtworker, greatone, gwapology, helphand, masteryoda, naughtyhrn, rgeddes, steemerhrn, steemfluencer, steemitwar, steemship, thovre, utfull]",150640,20.315577,22850.837,3.081704,60.091301,445577,steemit.com


### Most authors

In [85]:
unique_urls.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
1587428,https://coinmarketcap.com/,543
3902651,https://steemit.com/~witnesses,438
1337319,http://www.steemit.com/~witnesses,373
3416614,https://steemit.com,346
2799313,https://pixabay.com,256
2799375,https://pixabay.com/,256
3403560,https://steem.io/SteemWhitePaper.pdf,255
1501326,https://bittrex.com/,247
6570958,"https://vimeo.com"">Vimeo</a>.</p>",244
3638618,https://steemit.com/images/steemit-share.png,224


### Most votes

In [84]:
unique_urls.sort_values('total_votes', ascending=False)[0:20][['url', 'total_votes']]

,url,total_votes
3902651,https://steemit.com/~witnesses,1017302
3870056,https://steemit.com/translation/@papa-pepper/operation-translation-now-under-construction-what-it-is-and-how-you-can-help,389094
2469299,https://img1.steemit.com/0x0/http://i.imgur.com/uVNBbyL.jpg,354155
3944014,https://steemitimages.com/0x0/https://steemitimages.com/DQmWJLBUEQKrsCFYNSpzTSiRPbR8AT45ZpwbjgCX5ucQaDN/image.png,273629
1337794,http://www.steemvoter.com,218367
6678452,"https://www.facebook.com/groups/397766613731804/"">-",194767
3447567,"https://steemit.com/@sirwinchester""><img",193435
3607200,"https://steemit.com/food/@knozaki2015/i-travel-the-world-part-25-cafe-viena-feat-the-worlds-best-sandwich-ny-times"">Restaurant",186393
3582896,"https://steemit.com/deutsch/@knozaki2015/steemperlen-1"">German",186234
3606950,"https://steemit.com/food/@knozaki2015/fantastic-seafood-paella-cooked-2-with-the-master-of-paella-gargon"">10",181076


### Avg vote with 10 or more authors

In [83]:
unique_urls.query('num_authors >= 10').sort_values('avg_votes', ascending=False)[0:50][['url', 'avg_votes']]

,url,avg_votes
1628158,https://discord.gg/S37bYma,901.145455
1337794,http://www.steemvoter.com,768.897887
6782762,https://www.peerhub.com/,682.867521
3895903,https://steemit.com/witness/@teamsteem/what-are-steem-witnesses-and-why-you-should-support-them,658.971429
3539248,https://steemit.com/buildteam/@buildteam/launching-the-buildteam-token,630.166667
3895397,https://steemit.com/witness-category/@reggaemuffin/witness-reggaemuffin,543.111111
3693222,https://steemit.com/minnowbooster/@minnowbooster/introducing-minnowbooster-beta,532.238095
3495723,https://steemit.com/almost-famous/@teamsteem/hello-steemit-coinmaketcap-com-introduced-me,523.034483
3415959,https://steemit.chat/channel/curie,516.731034
3807338,https://steemit.com/steemit/@donkeypong/announcing-project-curie-bringing-rewards-and-recognition-to-steemit-s-undiscovered-and-emerging-authors,502.166667


### Total payout

In [36]:
unique_urls.sort_values('total_payout', ascending=False)[0:50][['url', 'total_payout']]

,url,total_payout
3902651,https://steemit.com/~witnesses,316353.439
3870056,https://steemit.com/translation/@papa-pepper/operation-translation-now-under-construction-what-it-is-and-how-you-can-help,87859.712
2469299,https://img1.steemit.com/0x0/http://i.imgur.com/uVNBbyL.jpg,84556.814
1113149,http://www.ericvancewalton.net,82858.708
3944014,https://steemitimages.com/0x0/https://steemitimages.com/DQmWJLBUEQKrsCFYNSpzTSiRPbR8AT45ZpwbjgCX5ucQaDN/image.png,81608.988
212093,"http://fuk.io""",71878.205
212181,http://fuk.io</a>,71878.205
3918363,"https://steemitimages.com/0x0/https://media.giphy.com/media/3ohzdPbKlUFVsmGpq0/giphy.gif""",71878.205
3415784,https://steemit.chat,67202.952
2479758,https://img1.steemit.com/0x0/https://www.dollarvigilante.com/wp-content/uploads/2016/08/Jeff-Berwick-The-Dollar-Vigilante-Steemit-Bio-1024x221.jpg</center>,59611.399


### Avg payout with 10 or more mentions

In [82]:
unique_urls.query('mentions >= 10').sort_values('avg_payout', ascending=False)[0:20][['url', 'total_payout']]

,url,total_payout
1337132,http://www.steemit.com/@steemdrive,37585.703
1800524,https://github.com/xeroc/piston,47905.119
1052447,http://www.bravenewcoin.com/steem,23633.304
1048460,http://www.blocktrades.us,21939.083
1118247,http://www.expanse.tech,20865.044
7138868,https://www.youtube.com/watch?v=YkJ7zxdpUwE,19667.129
53860,http://anarchapulco.com,25368.469
3538203,https://steemit.com/books/@federicopistono/hi-steem-community-i-am-the-author-of-robots-will-steal-your-job-but-thats,18287.280
473821,"http://i.imgur.com/hX8LV01.png""",15688.495
3823427,https://steemit.com/steemit/@teamsteem/a-bit-about-steemit,13278.872


## Websites

### Most mentioned

In [41]:
unique_domains.sort_values('mentions', ascending=False).head()

,domain,num_authors,mentions,authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation
168205,steemitimages.com,68390,3078718,"[a-a-a, a-a-lifemix, a-alice, a-angel, a-ayman, a-b, a-chixywilson, a-condor, a-jeffrey, a-k11u, a-kopf, a-league, a-m-s, a-sakura83, a-vm, a-vytimy, a10, a1000eyes, a13x, a1848art, a186r, a1a, a1ph4, a1sa, a2h1ey, a2k17, a2zmedia, a3abghani, a3ar, a4illusionist, a6r6e6k, a7mednabi7, a7x, a9933, aa-gudzini, aa-maryjblog, aa2nature, aa9, aaabbb, aaagent, aaalejandro, aaalex, aaanderson, aaaqib, aaasports, aaatriplethreat, aaav535, aabb, aabgina, aabidemi, aabir, aabir2060, aabisteemvoter, aace234, aachim63, aadarsh, aadcoin27, aadesh.abhang, aadi, aadi0786, aadiee, aadyashakti, aafeng, aafiya, aafrin, aaina, aajiz, aakanksh, aakash121, aakashagain, aakashsinha, aakashtyagi, aakii, aakzaki, aalagenesis, aalcardekza, aalihassan8, aaliyahholt, aaliyakhan, aamir, aamir135, aamirahmad, aamiralibhatti, aamirhussain, aamirhussainshah, aamirkhan, aamiryaqoob, aamna, aamnah, aams2329, aanandj5ee, aandi, aanfauzan, aankhlee007, aanrizkan, aanswer77, aanundsendk, aaqib11b, aaqib671, aaqibsohail, ...]",90378469,29.355878,2.242219e+07,7.282962,49.083123,151113093
168149,steemit.com,20290,1581770,"[a-a-a, a-a-lifemix, a-alice, a-angel, a-chixywilson, a-condor, a-eye, a-jeffrey, a-m-s, a-vytimy, a11at, a1848art, a1sa, a4illusionist, a9inchcock, aaabbb, aaagent, aaanderson, aace234, aadyashakti, aafeng, aajajim, aakashsinha, aakzaki, aarauz, aarcee, aarellanes, aariz11858, aarnold, aaronag, aaronaugustine, aaronburt, aaronc88, aaronhibberd, aaronjarnigan, aaronkoenig, aaronli, aaronmda, aaronpake, aaronparker, aaronsthilaire, aaronsuncamacho, aartijohn20, aaseb, aashishkandel, aashraf, abarefootpoet, abasifreke, abasinkanga, abbijulie, abcde, abcdoctor, abcteacher, abcxyzpo, abdel-ali, abdelhadi, abdelhamidtsouli, abdelilahelkati, abdellkarim, abdelrahmanhsn, abdelsamad10, abderrachidhafsi, abderrahiim, abderrahim, abdessamia, abdlhakim, abdo22, abduhawab, abdularhman, abdulawal39, abdullah001, abdullahmushtaq, abdullar, abduls, abegail, abejorro, abeljc, abelvangijlswijk, abenike, abh12345, abhi3700, abhilashrai, abhinav.badola, abhirockon, abhishek1, abhishekaggarwal, abhishekrajbhar, abid2017, abigail-dantes, abil, ability, abiolagbode, abiproud, abir20129, abishai, abit, abitary467, abitowhit, able, abloveshorchata, ...]",127203012,80.418147,4.348265e+07,27.489868,61.051277,96569079
203077,youtube.com,30941,409707,"[a-a-a, a-a-lifemix, a-angel, a-chixywilson, a-eye, a-hitler, a-jeffrey, a-m-s, a-m3001, a11at, a13xz, a1848art, a3ar, a4illusionist, a4v, a724, a9933, aaagent, aaalfonz, aaanderson, aaasports, aabgina, aabir, aace234, aachim63, aadarshpahilani, aadarshraj, aadyashakti, aakashsinha, aaliyahholt, aamiey, aamirhamza317, aanandj5ee, aaqibsohail, aarauz, aarellanes, aargy, aariman, aarkay, aarnold, aaronag, aaronb, aaronburt, aaronclarey, aaronepr, aaronfisher, aaronjarnigan, aaronjayjack, aaronkoenig, aaronli, aaronmda, aaronnn47, aaronpake, aaronsthilaire, aaront, aartijohn20, aaseb, aashish, aashishkandel, aashraf, aassddf, aassilakhalid, aaxlejh, aayalac22, aazaman9, ababeel, abad09, abagendo, abahous, abakazam, aballarde53, abandon, abaraza, abarefootpoet, abarilo, abas55, abatedwombat, abbasfxtm, abbeymelchizedek, abbzykhan, abc521, abcdoctor, abdallah118, abdallahfathy, abdel-ali, abdelhameedqotb, abdelhamidtsouli, abdelhasabiry, abdelilahelkati, abdelkader, abdellah, abdellahzaabat, abdellkarim, abdelsamad10, abderahim, abderrachidhafsi, abderrahiim, abderrahim, abderrahim1, abdesattar23, ...]",11479588,28.019018,3.826271e+06,9.339043,47.543742,19479004
90485,imgur.com,9854,407066,"[a-a-lifemix, a-alice, a-human, a9inchcock, aabgina, aakiswak, aakzaki, aarauz, aarkay, aaronb, aaronhoffman, aaronjarnigan, aaronli, aaronmda, aaronsthilaire, aaronsuncamacho, aaronx, aartijohn20, aazaman9, abanghafiz, abangsteem, abanks1000, abarefootpoet, abbyrose1214, abbyv, abcdoctor, abdal

### Most authors

In [81]:
unique_domains.sort_values('num_authors', ascending=False)[['domain', 'num_authors']][0:50]

,domain,num_authors
168205,steemitimages.com,68390
203077,youtube.com,30941
168149,steemit.com,20290
203066,youtu.be,16894
90485,imgur.com,9854
65008,facebook.com,9818
198358,wikipedia.org,8166
62758,esteem.ws,7583
186928,twitter.com,7297
199830,wordpress.com,6612


### Most votes

In [26]:
unique_domains.sort_values('total_votes', ascending=False).head()

,domain,mentions,authors,num_authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation
608,steemit.com,679,"[abit, au1nethyb1, clayop, dantheman, dele-pup...",24,140318,206.653903,141706.361,208.698617,68.680412,46634
341,imgur.com,470,"[abit, betax, clayop, dele-puppy, donalddrumpf...",15,139206,296.182979,84831.924,180.493455,67.944681,31934
93,bitsharestalk.org,243,"[abit, clayop, ihashfury, nextgencrypto, phare...",10,82219,338.349794,35089.596,144.401630,71.489712,17372
609,steemitimages.com,353,"[abit, clayop, dantheman, ned, pharesim, roads...",10,74873,212.104816,78510.476,222.409280,67.716714,23904
277,github.com,261,"[abit, au1nethyb1, betax, clayop, dantheman, f...",22,48010,183.946360,149800.654,573.948866,67.360153,17581


### Avg vote with with 10 or more mentions

In [50]:
unique_domains.query('num_authors >= 10').sort_values('num_authors', ascending=False)[['domain', 'num_authors']][0:50]

,domain,num_authors
168205,steemitimages.com,68390
203077,youtube.com,30941
168149,steemit.com,20290
203066,youtu.be,16894
90485,imgur.com,9854
65008,facebook.com,9818
198358,wikipedia.org,8166
62758,esteem.ws,7583
186928,twitter.com,7297
199830,wordpress.com,6612


### Total payout

In [28]:
unique_domains.sort_values('total_payout', ascending=False).head()

,domain,mentions,authors,num_authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation
340,imgsafe.org,100,"[au1nethyb1, clayop, coin, ihashfury, kushed, ...",12,31024,310.240000,330268.867,3302.688670,67.200000,6720
517,postimg.org,140,"[beyondbitcoin, clayop, coin, dantheman, jabba...",12,37169,265.492857,156314.307,1116.530764,69.278571,9699
277,github.com,261,"[abit, au1nethyb1, betax, clayop, dantheman, f...",22,48010,183.946360,149800.654,573.948866,67.360153,17581
608,steemit.com,679,"[abit, au1nethyb1, clayop, dantheman, dele-pup...",24,140318,206.653903,141706.361,208.698617,68.680412,46634
341,imgur.com,470,"[abit, betax, clayop, dele-puppy, donalddrumpf...",15,139206,296.182979,84831.924,180.493455,67.944681,31934


### Avg payout with 10 or more mentions

In [29]:
unique_domains.query('mentions >= 10').sort_values('avg_payout', ascending=False).head()

,domain,mentions,authors,num_authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation
340,imgsafe.org,100,"[au1nethyb1, clayop, coin, ihashfury, kushed, ...",12,31024,310.240000,330268.867,3302.688670,67.200000,6720
517,postimg.org,140,"[beyondbitcoin, clayop, coin, dantheman, jabba...",12,37169,265.492857,156314.307,1116.530764,69.278571,9699
149,coinigy.com,26,steempower,1,4185,160.961538,22146.081,851.772346,72.000000,1872
620,streemian.com,12,"[au1nethyb1, xeroc]",2,1291,107.583333,8428.155,702.346250,69.333333,832
277,github.com,261,"[abit, au1nethyb1, betax, clayop, dantheman, f...",22,48010,183.946360,149800.654,573.948866,67.360153,17581


### Top 20 websites by author steem power

### Top 20 websites by log of author steem power

## Twitter

In [51]:
domain = 'twitter.com'

In [58]:
unique_urls[unique_urls.domain == domain].head()

,url,mentions,authors,num_authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation,domain
228,http://www.twitter.com/steemservices,2,steemservices,1,196,98.0,1568.969,784.4845,65.0,130,twitter.com
472,https://twitter.com/EL/status/732374086580461568,1,jabbasteem,1,1,1.0,0.110,0.1100,53.0,53,twitter.com
473,https://twitter.com/SteemServices,1,steemservices,1,21,21.0,68.744,68.7440,65.0,65,twitter.com
512,https://www.twitter.com/steemservices,2,steemservices,1,75,37.5,486.058,243.0290,65.0,130,twitter.com


### Most Mentioned Status's

In [66]:
twitter_status = unique_urls[(unique_urls.domain == domain) & (unique_urls.url.str.contains('/status/'))]
twitter_status.sort_values('num_authors', ascending=False)[0: 50][['url', 'num_authors']]

,url,num_authors
3788219,https://steemit.com/status/@cryptoblood/ico-price-alert-status-now-trading-below-ico-price-and-128201,1
3788220,https://steemit.com/status/@sona4ajay/happy-today#@kekhrie/re-sona4ajay-happy-today-20170911t070419147z,1


### Most Mentioned Accounts

In [67]:
twitter_status = unique_urls[(unique_urls.domain == domain) & (unique_urls.url.str.count('/') == 3)]
twitter_status.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
3902651,https://steemit.com/~witnesses,438
1337319,http://www.steemit.com/~witnesses,373
3416706,https://steemit.com/,220
3595174,https://steemit.com/faq.html,88
3438166,https://steemit.com/@luzcypher,79
3892765,https://steemit.com/welcome,77
3862628,https://steemit.com/tags,77
3687618,https://steemit.com/market,72
3426064,https://steemit.com/@dollarvigilante,56
3433420,https://steemit.com/@jerrybanfield,45


## Steemit.com

In [59]:
domain = 'steemit.com'

In [75]:
unique_urls[unique_urls.domain == domain].head()

,url,mentions,authors,num_authors,total_votes,avg_votes,total_payout,avg_payout,avg_reputation,total_reputation,domain
188,http://steemit.com,4,"[steemed, xeldal]",2,143,35.75,867.238,216.8095,62.25,249,steemit.com
189,http://steemit.com/,1,nextgencrypto,1,12,12.00,14.646,14.6460,65.00,65,steemit.com
424,https://steemit.com/,5,"[ihashfury, steemed]",2,172,34.40,1550.537,310.1074,58.80,294,steemit.com
425,https://steemit.com/@complexring,1,au1nethyb1,1,7,7.00,0.016,0.0160,62.00,62,steemit.com
426,https://steemit.com/@jabbasteem,1,steemservices,1,77,77.00,603.192,603.1920,65.00,65,steemit.com


### Most mentioned Steemit Urls

In [69]:
steemit_urls = unique_urls[(unique_urls.domain == domain)]
steemit_urls.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
3902651,https://steemit.com/~witnesses,438
1337319,http://www.steemit.com/~witnesses,373
3416614,https://steemit.com,346
3638618,https://steemit.com/images/steemit-share.png,224
3416706,https://steemit.com/,220
3890329,https://steemit.com/video/@heimindanger/introducing-dtube-a-decentralized-video-platform-using-steem-and-ipfs,101
3638620,"https://steemit.com/images/steemit-share.png""",91
3595174,https://steemit.com/faq.html,88
896056,http://steemit.com,86
2474299,https://img1.steemit.com/0x0/https://i.imgsafe.org/b87bb98157.gif,83


### Most mentioned Steemit Accounts

In [80]:
steemit_accounts = unique_urls[(unique_urls.domain == domain) & (unique_urls.url.str.contains('.com/@'))]
steemit_accounts.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
1753402,https://facebook.com/@exscudo,1
2634853,"https://l.facebook.com/l.php?u=https%3A%2F%2Fsteemit.com%2F%40allucky111&amp;h=ATN3UiHyV6rPg5rHDRt2Kwe_Ma1Ep7GCK5w_Q7NZ7CRSjtW-zY9bTkOr3YvTuCYJt06_RWvq1G8rUvlanNavt2lp7bs0DFfuJ6Y10m1fQxfN-tGEgLlUI0H7oGdImNNv53W7s-oFZoC0sbNrVz0siD1v&amp;enc=AZMUgVgxIU5FiHbjJ4gpiunqdKZfiPuetdHm7lWAdPEyDyaBrRKD-NIzH-ndKxi_6QOEvODgbYx1yt-oGq9Wr6L-gCEq8E4cOAn9BRdEAP2pMQ0BOFQqtdBKn-GKtGO0lGR9LYgILXGQN1NOJi44rO-fvM60s8eF4mNoua1KVe-sCNSe9cW2c0By-k-pLopoBqu9U2KJ-u-7aVg2I23MPsHp"">https://steemit.com/@allucky111</a>&nbsp;</p>",1
2634854,"https://l.facebook.com/l.php?u=https%3A%2F%2Fsteemit.com%2F%40manann&amp;h=ATMFox9n80WC9t6-ZNfhIJwLfajYt5hi1Thb4yyHZa72hd3sRsDBo7U3g3r-zsyif27DGYY4ruY80AlFa9HTRb1BOueWRufafQncbfl-lPNsPK3vAbBONQui8r2hVE0Dw5y3XsVDEGWS&amp;s=1"">https://steemit.com/@manann</a>",1
2634918,"https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.steemit.com%2F%40maryfavour%2F&amp;h=ATPEnj8rs27qppE3MjN-zaRe8oAWLproUiY0oLHnGgIFMC0gvUWUulJt7WOO-Hk6VLEvkDPUt46bRXBYKnMWAa_OZ6L_ZgRJfAVX2Xz2ZPJ6vN52ViMbXPigtHmNntpdq8bY_S4LP08"">https://www.steemit.com/@maryfavour/</a>",1
6666376,https://www.facebook.com/@nazmul,1


## Facebook

In [72]:
domain = 'facebook.com'

### Most mentioned Facebook Urls

In [87]:
facebook_urls = unique_urls[(unique_urls.domain == domain)]
facebook_urls.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
1753741,https://facebook.com/steemitmarket,62
6665912,https://www.facebook.com/,36
6667548,https://www.facebook.com/CCSer/,33
6675498,https://www.facebook.com/cosscrypto/,26
6674737,https://www.facebook.com/blocklancer,26
6678880,https://www.facebook.com/groups/ICoLR/,19
6688118,https://www.facebook.com/utrust.io/,18
6687030,https://www.facebook.com/steemit/,17
6669485,https://www.facebook.com/InsureX-1467549329975479,17
6686116,"https://www.facebook.com/rsrc.php/v3/y4/r/-PAXP-deijE.gif""",17


### Most mentioned Facebook usernames

In [78]:
facebook_urls = unique_urls[(unique_urls.domain == domain) & (unique_urls.url.str.count('/') == 3)]
facebook_urls.sort_values('num_authors', ascending=False)[0:50][['url', 'num_authors']]

,url,num_authors
1753741,https://facebook.com/steemitmarket,62
6665912,https://www.facebook.com/,36
6674737,https://www.facebook.com/blocklancer,26
6669485,https://www.facebook.com/InsureX-1467549329975479,17
6670281,https://www.facebook.com/Mandarin.exchange,14
6682953,https://www.facebook.com/nimfamoney,14
6677023,https://www.facebook.com/exscudo,13
6667678,https://www.facebook.com/CentraCard,13
6675890,https://www.facebook.com/dentcoin,11
6687278,https://www.facebook.com/taasfund,10
